In [1]:
import pandas as pd

In [2]:
train=pd.read_csv('exoTrain.csv')

In [3]:
test=pd.read_csv('exoTest.csv')

In [4]:
train.head()

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,2,93.85,83.81,20.10,-26.98,-39.56,-124.71,-135.18,-96.27,-79.89,...,-78.07,-102.15,-102.15,25.13,48.57,92.54,39.32,61.42,5.08,-39.54
1,2,-38.88,-33.83,-58.54,-40.09,-79.31,-72.81,-86.55,-85.33,-83.97,...,-3.28,-32.21,-32.21,-24.89,-4.86,0.76,-11.70,6.46,16.00,19.93
2,2,532.64,535.92,513.73,496.92,456.45,466.00,464.50,486.39,436.56,...,-71.69,13.31,13.31,-29.89,-20.88,5.06,-11.80,-28.91,-70.02,-96.67
3,2,326.52,347.39,302.35,298.13,317.74,312.70,322.33,311.31,312.42,...,5.71,-3.73,-3.73,30.05,20.03,-12.67,-8.77,-17.31,-17.35,13.98
4,2,-1107.21,-1112.59,-1118.95,-1095.10,-1057.55,-1034.48,-998.34,-1022.71,-989.57,...,-594.37,-401.66,-401.66,-357.24,-443.76,-438.54,-399.71,-384.65,-411.79,-510.54


In [5]:
test.head()

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,2,119.88,100.21,86.46,48.68,46.12,39.39,18.57,6.98,6.63,...,14.52,19.29,14.44,-1.62,13.33,45.50,31.93,35.78,269.43,57.72
1,2,5736.59,5699.98,5717.16,5692.73,5663.83,5631.16,5626.39,5569.47,5550.44,...,-581.91,-984.09,-1230.89,-1600.45,-1824.53,-2061.17,-2265.98,-2366.19,-2294.86,-2034.72
2,2,844.48,817.49,770.07,675.01,605.52,499.45,440.77,362.95,207.27,...,17.82,-51.66,-48.29,-59.99,-82.10,-174.54,-95.23,-162.68,-36.79,30.63
3,2,-826.00,-827.31,-846.12,-836.03,-745.50,-784.69,-791.22,-746.50,-709.53,...,122.34,93.03,93.03,68.81,9.81,20.75,20.25,-120.81,-257.56,-215.41
4,2,-39.57,-15.88,-9.16,-6.37,-16.13,-24.05,-0.90,-45.20,-5.04,...,-37.87,-61.85,-27.15,-21.18,-33.76,-85.34,-81.46,-61.98,-69.34,-17.84


In [8]:
train['LABEL'].unique()

array([2, 1], dtype=int64)

In [9]:
x_train=train.drop('LABEL',axis=1)

In [10]:
y_train=train.LABEL-1

In [11]:
x_test=test.drop('LABEL',axis=1)

In [12]:
y_test=test.LABEL-1

In [14]:
#saving names of all features in a list

In [15]:
numeric_column_headers=x_train.columns.values.tolist()

In [20]:
#bucketizing the feature column into two buckets around the mean since the TFBT estimator only takes bucketed featuers

In [22]:
import tensorflow as tf

In [23]:
bc_fn=tf.feature_column.bucketized_column
nc_fn=tf.feature_column.numeric_column

In [25]:
bucketized_features=[bc_fn(source_column=nc_fn(key=column),boundaries=[x_train[column].mean()]) for column in numeric_column_headers]

In [27]:
all_features=bucketized_features

In [31]:
#input functions

In [29]:
batch_size=32
pi_fn=tf.estimator.inputs.pandas_input_fn
train_input_fn=pi_fn(x=x_train,y=y_train,batch_size=batch_size,shuffle=True,num_epochs=None)

In [30]:
eval_input_fn=pi_fn(x=x_test,y=y_test,batch_size=batch_size,shuffle=False,num_epochs=1)

In [32]:
#choosing model parameters

In [33]:
n_trees=100
n_steps=100

In [34]:
m_fn=tf.estimator.BoostedTreesClassifier

In [35]:
model=m_fn(feature_columns=all_features,n_trees=n_trees,n_batches_per_layer=batch_size,model_dir='./tfbtmodel')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './tfbtmodel', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001A23CCF3E48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [36]:
model.train(input_fn=train_input_fn,steps=n_steps)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving checkpoints for 0 into ./tfbtmodel\model.ckpt.
Type is unsupported, or the types of the items don't match field type in Colle

In [37]:
results=model.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-01-16:41:28
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tfbtmodel\model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-01-16:41:45
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.99122804, accuracy_baseline = 0.99122804, auc = 0.6771682, auc_precision_recall = 0.01895111, average_loss = 0.46445137, global_step = 100, label/mean = 0.00877193, loss = 0.4644097, precision = 0.0, prediction/mean = 0.368616, recall = 0.0
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100: ./tfbtmodel\model.ckpt-100


In [45]:
for key,value in sorted(results.items()):
    print('{}: {}'.format(key,value))

accuracy: 0.9912280440330505
accuracy_baseline: 0.9912280440330505
auc: 0.6771681904792786
auc_precision_recall: 0.018951110541820526
average_loss: 0.4644513726234436
global_step: 100
label/mean: 0.008771929889917374
loss: 0.4644097089767456
precision: 0.0
prediction/mean: 0.3686160147190094
recall: 0.0
